In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Nov 15 11:41:48 2017

@author: WangJin_XuHan
"""
import dlib
import matplotlib.pyplot as plt
from matplotlib.pylab import datestr2num
import pandas as pd

import math
import tensorflow as tf
import cv2
import numpy as np
import os
import random
import sys
from sklearn.model_selection import train_test_split

### 1. Prepare the data and then read them

In [6]:
my_faces_path = 'D:/Desktop/InfoProject/face_jp/my_final'
other_faces_path = 'D:/Desktop/InfoProject/face_jp/noise_final'
size = 64

imgs = []
# labs = []
pathes = []

In [7]:
#Here make the input image as square
def getPaddingSize(img):
    h, w, _ = img.shape
    top, bottom, left, right = (0,0,0,0)
    longest = max(h, w)

    if w < longest:
        tmp = longest - w
        # // is Floor Division; 9//2 = 4
        left = tmp // 2
        right = tmp - left
    elif h < longest:
        tmp = longest - h
        top = tmp // 2
        bottom = tmp - top
    else:
        pass
        
    return top, bottom, left, right

In [8]:
# def readData(path, h=size, w=size):
def readData(path):
    for filename in os.listdir(path):    #get names of files or folders
        if filename.endswith('.jpg'):
            filename = path + '/' + filename
            
            #read image with cv2
            img = cv2.imread(filename)               
            #the four means space to fill on which direction to make image square
            top,bottom,left,right = getPaddingSize(img)        
            # enlare the picture, filled the edges with zeros
            img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0,0,0])            
            
            #we can resize here, but no not need to, because our data has been resized
          
            #create matrix array
            imgs.append(img)            
            #useful storage for the following step creating lables 
            pathes.append(path)

In [9]:
#read data and split the data set
readData(my_faces_path)
readData(other_faces_path)

# append matrix array (from picture) in to array; not that necessary but good 
imgs = np.array(imgs)
# creat lable array for supervised learning; #one line ifelse makes python looks like pseudo code
labs = np.array([[0,1] if lab == my_faces_path else [1,0] for lab in pathes])   

# split the training data set and test dataset; test_ratio is ration of test data set
test_ratio = 0.3
train_x,test_x,train_y,test_y = train_test_split(imgs, labs, test_size=test_ratio, random_state=random.randint(0,100))

# number of pics,hight,width,3 means color picture
train_x = train_x.reshape(train_x.shape[0], size, size, 3)
test_x = test_x.reshape(test_x.shape[0], size, size, 3)

#Normalization
train_x = train_x.astype('float32')/255.0
test_x = test_x.astype('float32')/255.0

print('Split completed! train size:%s, test size:%s' % (len(train_x), len(test_x)))

#batch is to avoid the ram jam
batch_size_read = 200
#get interger using ceiling
num_batch_train = math.ceil(len(train_x) / batch_size_read)
num_batch_test = math.ceil(len(test_x) / batch_size_read) 


x = tf.placeholder(tf.float32, [None, size, size, 3])
y_ = tf.placeholder(tf.float32, [None, 2])


#drop_out hence avoid overfitting
keep_prob_5 = tf.placeholder(tf.float32)
keep_prob_75 = tf.placeholder(tf.float32)

Split completed! train size:16100, test size:6900


### 2. Build the CNN network with tensorflow

In [26]:
#paprameter for the different layers
def weightVariable(shape):
    init = tf.random_normal(shape, stddev=0.01)
    return tf.Variable(init)

def biasVariable(shape):
    init = tf.random_normal(shape)
    return tf.Variable(init)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')


def maxPool(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

def dropout(x, keep):
    return tf.nn.dropout(x, keep)

In [27]:
def cnnLayer():

    #first layer
    W1 = weightVariable([3,3,3,32]) # filter(3,3)， input channel(3)， output channel(32)
    b1 = biasVariable([32])
    #concoluntion layer
    conv1 = tf.nn.relu(conv2d(x, W1) + b1)
    #pooling layer
    pool1 = maxPool(conv1)
    # dropout is to avoid fitting
    drop1 = dropout(pool1, keep_prob_5)

    # second layer
    W2 = weightVariable([3,3,32,64])
    b2 = biasVariable([64])
    conv2 = tf.nn.relu(conv2d(drop1, W2) + b2)
    pool2 = maxPool(conv2)
    drop2 = dropout(pool2, keep_prob_5)

    # third layer
    W3 = weightVariable([3,3,64,64])
    b3 = biasVariable([64])
    conv3 = tf.nn.relu(conv2d(drop2, W3) + b3)
    pool3 = maxPool(conv3)
    drop3 = dropout(pool3, keep_prob_5)

    # full connection layer1
    Wf = weightVariable([8*16*32, 512])
    bf = biasVariable([512])
    drop3_flat = tf.reshape(drop3, [-1, 8*16*32])
    dense = tf.nn.relu(tf.matmul(drop3_flat, Wf) + bf)
    dropf = dropout(dense, keep_prob_75)

    # output layer
    Wout = weightVariable([512,2])
    #bout = weightVariable([2])
    bout = biasVariable([2])
    #out = tf.matmul(dropf, Wout) + bout
    out = tf.add(tf.matmul(dropf, Wout), bout)
    return out

In [28]:
def cnnTrain(epoch_num=20, batch_size=200):
    #call cnnLayer to build architecture
    out = cnnLayer()
    
    #Here we use cross entropy as loss fuction
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=y_))
    
    #train the model using Ada#m Optimizer
    train_step = tf.train.AdamOptimizer(0.005).minimize(cross_entropy)
    
    #prepare the accuracy evaluation
    correct_prediction = tf.equal(tf.argmax(out, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    #this is to help the calculate the accuracy
    acc_sum = 0
    
    #prepare the visualization using matplot line plot   
    #use pandas dataframe to store the (#epoh,accuracy)
    accu_df = pd.DataFrame(np.random.randint(low=0, high=1, size=(epoch_num, 1)),columns=['accuracy']) 

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(epoch_num):
            for i in range(num_batch_train):
                batch_x = train_x[i*batch_size : (i+1)*batch_size]
                batch_y = train_y[i*batch_size : (i+1)*batch_size]

                # start train! exciting!
                _,loss = sess.run([train_step, cross_entropy],
                                          feed_dict={x:batch_x,y_:batch_y, keep_prob_5:0.5,keep_prob_75:0.75})
            print('Awesome! Completed No. %d epoch of training!'%(epoch))            

        
        #test model part
            for j in range(num_batch_test):
                batch_test_x = test_x[j*batch_size : (j+1)*batch_size]
                batch_tset_y = test_y[j*batch_size : (j+1)*batch_size]

                acc = accuracy.eval({x:batch_test_x, y_:batch_tset_y, keep_prob_5:1.0, keep_prob_75:1.0})

                #print("%d batch's accuracy: %f"%((j+1),acc))

            #calculate the mean
            acc_sum += acc
            acc_mean = acc_sum/(epoch+1)
            print('So far accuracy: %f'%acc_mean) 
            
            #store the accuracy into the dataframe(table)
            accu_df.loc[epoch] = acc_mean

    print('Finish! You are genius!')
 
    return accu_df

In [29]:
#main function
epoch_num=20
batch_size=200

#call main function
accu_epoch = cnnTrain(epoch_num, batch_size)
accu_epoch

Awesome! Completed No. 0 epoch of training!
So far accuracy: 0.900000
Awesome! Completed No. 1 epoch of training!
So far accuracy: 0.910000
Awesome! Completed No. 2 epoch of training!
So far accuracy: 0.930000
Awesome! Completed No. 3 epoch of training!
So far accuracy: 0.942500
Awesome! Completed No. 4 epoch of training!
So far accuracy: 0.952000
Awesome! Completed No. 5 epoch of training!
So far accuracy: 0.958333
Awesome! Completed No. 6 epoch of training!
So far accuracy: 0.962857
Awesome! Completed No. 7 epoch of training!
So far accuracy: 0.963750
Awesome! Completed No. 8 epoch of training!
So far accuracy: 0.964444
Awesome! Completed No. 9 epoch of training!
So far accuracy: 0.966000
Awesome! Completed No. 10 epoch of training!
So far accuracy: 0.968182
Awesome! Completed No. 11 epoch of training!
So far accuracy: 0.970000
Awesome! Completed No. 12 epoch of training!
So far accuracy: 0.970769
Awesome! Completed No. 13 epoch of training!
So far accuracy: 0.972857
Awesome! Complet

,accuracy
0,0.900000
1,0.910000
2,0.930000
3,0.942500
4,0.952000
5,0.958333
6,0.962857
7,0.963750
8,0.964444
9,0.966000


### 3.Visualize the training process

In [ ]:
#store the result
accu_epoch.to_csv('final_test.csv',index=False,header=True)

#read accuracy data and visualize it
final_test = pd.read_csv('final_test.csv')

#combine the accuracy tables  
accu_epoch =pd.concat([final_test], axis=1)
print(accu_epoch)

#plot the combines table
accu_epoch.plot( kind='line',title='accuracy along with epoches(batch size 200)',grid=True,legend=True,figsize=(12,8))

plt.savefig('accuracy_final.jpg')
plt.show()